In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

import sys
import os
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [66]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

from project.datasets import Dataset, CTRPDataModule
from project.film_model import FiLMNetwork, ConcatNetwork

In [3]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from sklearn import model_selection
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping

In [30]:
import pyarrow.dataset as ds
import pyarrow.feather as feather

In [23]:
data = ds.dataset(data_path.joinpath("train_sub.feather"), format='feather')

In [55]:
def prepare(exp, subset=True):
    data_path = Path("../../film-gex-data/processed/")
    input_cols = joblib.load(data_path.joinpath("gene_cols.pkl"))
    
    if exp=='id':
        cpd_id = "master_cpd_id"
        cond_cols = np.array([cpd_id, 'cpd_conc_umol'])
    else:
        fp_cols = joblib.load(data_path.joinpath("fp_cols.pkl"))
        cond_cols = np.append(fp_cols, ['cpd_conc_umol'])
        
    if subset:
        dataset = ds.dataset(data_path.joinpath("train_sub.feather"), format='feather')
    else:
        dataset = ds.dataset(data_path.joinpath("train.feather"), format='feather')
        
    return dataset, input_cols, cond_cols


In [79]:
def cv(name, exp, nfolds, dataset, input_cols, cond_cols, batch_size):
    seed_everything(2299)
    cols = list(np.concatenate((input_cols, cond_cols, ['cpd_avg_pv'])))
    
    for fold in np.arange(0,nfolds):
        train = dataset.to_table(columns=cols, filter=ds.field('fold') != fold).to_pandas()
        val = dataset.to_table(columns=cols, filter=ds.field('fold') == fold).to_pandas()
        # DataModule
        dm = CTRPDataModule(train,
                            val,
                            input_cols,
                            cond_cols,
                            target='cpd_avg_pv',
                            batch_size=batch_size)
        # Model
        if exp=='film':
            model = FiLMNetwork(len(input_cols), len(cond_cols))
        else:
            model = ConcatNetwork(len(input_cols), len(cond_cols))
        # Callbacks
        logger = TensorBoardLogger(save_dir=os.getcwd(),
                                   version="{}_{}_fold_{}".format(name, exp, fold),
                                   name='lightning_logs')
        early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=0.01)
        # Trainer
        trainer = Trainer(max_epochs=15, 
                          gpus=[1,2,3,4,5,6],
                          logger=logger,
                          early_stop_callback=early_stop,
                          distributed_backend='dp')
        trainer.fit(model, dm)
    return print("Completed CV")

In [80]:
dataset, input_cols, cond_cols = prepare('id', subset=True)

In [81]:
name = 'test'
exp = 'id'
nfolds = 1

In [82]:
cv(name, exp, nfolds, dataset, input_cols, cond_cols, batch_size=256)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [1,2,3,4,5,6]
/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name       | Type        | Params
-------------------------------------------
0 | metric     | R2Score     | 0     
1 | inputs_emb | LinearBlock | 677 K 
2 | conds_emb  | LinearBlock | 96    
3 | block_1    | LinearBlock | 1 K   
4 | block_2    | LinearBlock | 161   


Epoch 1:  32%|███▏      | 720/2254 [01:15<02:41,  9.50it/s, loss=0.092, v_num=ld_0]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe6a2e00670>
Traceback (most recent call last):
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 929, in _shutdown_workers
    self._pin_memory_thread.join()
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/threading.py", line 1011, in join
    self._wait_for_tstate_lock()
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/threading.py", line 1027, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt: 
/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)

Epoch 1:  32%|███▏      | 720/2254 [01:16<02:43,  9.40it/s, loss=0.092, v_num=ld_0]
Completed CV


Exception in thread Thread-13539:
Traceback (most recent call last):
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 294, in rebuild_storage_fd
    fd = df.detach()
  File "/srv/home/wconnell/anaconda3/envs/lightning/lib/python3.8/multiprocessing/resource_sharer.py", line 58, in detach
    return redu

In [15]:
%tensorboard --logdir ./lightning_logs --port 9066

In [22]:
trainer.default_root_dir

'/srv/home/wconnell/github/film-gex/notebooks'

In [8]:
import torch

In [9]:
foo = torch.FloatTensor(val[cond_cols].to_numpy())
model.forward(foo)

tensor([[ -5.2721,  -5.0805,   1.7124,  ...,  -5.4074,   1.7200, -13.4092],
        [ -0.8518,   0.4557,  -1.1743,  ...,   0.7598,   1.3746,  -2.2434],
        [ -1.5418,   1.0390,  -1.1317,  ...,   1.3517,   2.7206,  -1.4814],
        ...,
        [ -1.4564,   1.6760,   0.2275,  ...,   1.7220,   1.8293,  -1.2928],
        [ -2.3295,  -1.4681,  -0.4503,  ...,  -2.5142,   2.1346,  -8.3439],
        [ -2.1074,  -1.6664,   0.6190,  ...,  -1.4646,   1.0616,  -6.0193]],
       grad_fn=<AddmmBackward>)

In [26]:
data.shape

(57676, 1502)

In [28]:
data[data['fold']==0].shape

(11474, 1502)

In [29]:
11474 / 512

22.41015625